### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Sean Wells smwells3

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class, office hours, or piazza. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model

## 1. Feature engineering (one-hot encoding and data imputation)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.

(We used these data before in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

In [2]:
titanic = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv")
titanic = titanic[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch"]]

In [3]:
titanic[:7]

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [4]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
#knn.fit(titanic[["Pclass", "Sex", "Age", "SibSp", "Parch"]], titanic["Survived"])

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [5]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
#knn.fit(titanic[["Pclass", "Age", "SibSp", "Parch"]], titanic["Survived"])
titanic["Age"].isna().any()

True

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [6]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
X = titanic[["Pclass", "SibSp", "Parch"]]
y = titanic["Survived"]
knn.fit(titanic[["Pclass", "SibSp", "Parch"]], titanic["Survived"])
print(f"Accuracy on training data is {knn.score(X,y):.3}.")

Accuracy on training data is 0.633.


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [7]:
dumtit = titanic.join(pd.get_dummies(titanic.Sex))
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
X = dumtit[["Pclass", "female", "SibSp", "Parch"]]
y = dumtit["Survived"]
knn.fit(X,y)
print(f"The training accuracy is {knn.score(X,y):.3}.")

The training accuracy is 0.744.


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [8]:
imp = SimpleImputer(missing_values=np.nan, strategy='median', fill_value=None)
titanic.Age = imp.fit_transform(titanic.Age.to_numpy().reshape(-1, 1))
dumtit = titanic.join(pd.get_dummies(titanic.Sex))
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
X = dumtit[["Pclass", "female", "SibSp", "Parch", "Age"]]
y = dumtit["Survived"]
knn.fit(X, y)
print(f"The training accuracy is {knn.score(X,y):.3}.")

The training accuracy is 0.861.


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [9]:
mtcars = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv")
X = mtcars.loc[:, mtcars.columns!="mpg"]
y = mtcars["mpg"]
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=.5,
                                                  random_state=0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [14]:
models = [linear_model.LinearRegression(),
              linear_model.Lasso(max_iter=2000),
              linear_model.Ridge()]
for model in models:
    model.fit(X_train.iloc[:,1:], y_train)
    train_mse = (1/y_train.size) * np.sum((y_train - model.predict(X_train.iloc[:,1:]))**2)
    test_mse = (1/y_tmp.size) * np.sum((y_tmp - model.predict(X_tmp.iloc[:,1:]))**2)
    print(f"The training and test mean squared errors of the {model} model are {train_mse:.3}, {test_mse:.3} respectively.")
    print(f"Model coefficients: {np.round(model.coef_,3)}\n\n")

The training and test mean squared errors of the LinearRegression() model are 0.386, 30.2 respectively.
Model coefficients: [ 1.082  0.025  0.026  3.13  -7.343  3.932 -4.086 -1.216  5.987 -0.771]


The training and test mean squared errors of the Lasso(max_iter=2000) model are 5.69, 13.0 respectively.
Model coefficients: [-0.    -0.037 -0.017  0.    -0.     0.     0.     0.     0.    -0.847]


The training and test mean squared errors of the Ridge() model are 1.99, 11.2 respectively.
Model coefficients: [-0.014 -0.007  0.004  0.793 -3.219  1.098 -0.482  0.471  1.495 -1.045]




### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?- 

The linear model best fits the training data, although the test mean squared error of this model tells us that it is extremely overfitted. Ridge regression performs best on test data by a slight margin compared to Lasso, in which Lasso uniquely does feature selection by setting coefficients to zero. 